<a href="https://colab.research.google.com/github/kimhj010502/ESAA_OB/blob/main/WEEK12_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b> [파이썬 머신러닝 완벽 가이드] </b> <br>
## Chapter 09 추천 시스템

## 9.8 파이썬 추천 시스템 패키지 - Surprise <br>
### Surprise 패키지 소개 <br>
- 다양한 추천 알고리즘. 예를 들어 사용자 또는 아이템 기반 최근접 이웃 협업 필터링. SVD, SVD++, NMF 기반의 잠재 요인 협업 필터링을 쉽게 적용해 추천 시스템을 구축할 수 있음
- Surprise의 핵심 API는 사이킷런의 핵심 API와 유사한 API명으로 작성되었음. 예를 들어 `fit()`, `predict()` API로 추천 데이터 학습과 예측, `train_test_split()`으로 추천 학습 데이터 세트와 예측 데이터 세트 분리, `cross_validate()`, GridSearchCV 클래스를 통해 추천 시스템을 위한 모델 셀렉션, 평가, 하이퍼 파라미터 튜닝 등의 기능을 제공함

### Surprise를 이용한 추천 시스템 구축 <br>

In [1]:
! pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 4.7 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633937 sha256=7b11a3cecf797ef579dff197f40f676fb3bc6abd280545c28caeca8edb253da0
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [4]:
data = Dataset.load_builtin('ml-100k', prompt=False)
#수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...


URLError: ignored

- Surprise는 자체적으로 로우 레벨의 데이터를 칼럼 레벨의 데이터로 변경하므로 원본인 로우 레벨의 사용자-아이템 평점 데이터를 데이터 세트로 적용해야 함


In [ ]:
algo = SVD() #알고리즘 객체 생성성
algo.fit(trainset)

- Surprise에서 추천을 예측하는 메소드: `test()`, `predict()`
    - `test()`: 사용자-아이템 평점 데이터를 세트 전체에 대해서 추천을 예측하는 메소드로, 입력된 데이터 세트에 대해 추천 데이터 세트를 만들어줌
    - `preidct()`: 개별 사용자와 영황에 대한 추천 평점을 반환해줌

In [ ]:
predictions = algo.test(testset)
print('prediction type:', type(predictions), ' size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

- Prediction 객체는 Surprise 패키지에서 제공하는 데이터 타입이며, 개별 사용자 아이디 (uid), 영화 (또는 아이템) 아이디 (iid)와 실제 평점 (r_ui) 정보에 기반해 Surprise의 추천 예측 평점(est) 데이터를 튜플 형태로 가지고 있음
- Prediction 객체의 details 속성은 내부 처리 시 추천 예측을 할 수 없는 경우에 로그용으로 데이터를 남기는 데 사용되며 'was_impossible'이 True이면 예측값을 생성할 수 없는 데이터라는 의미임

In [ ]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

- `predict()`는 개별 사용자의 아이템에 대한 추천 평점을 예측해 줌
- 인자로 개별 사용자 아이디, 아이템 아이디를 입력하면 추천 예측 평점을 포함한 정보를 반환함
- 기준 평점 정보는 선택 사항이며 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함

In [ ]:
#사용자 아이디, 아이템 아이디는 문자열로 입력해야 함
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

- `predict()`는 개별 사용자와 아이템 정보를 입력하면 추천 예측 평점을 est로 반환함
- `test()` 메소드는 입력 데이터 세트의 모든 사용자와 아이템 아이디에 대해서 `predict()`를 반복적으로 수행한 결과임 

In [ ]:
accuracy.rmse(predictions)

### Surprise 주요 모듈 소개 <br>
#### Dataset
- Surprise는 user_id, item_id, rating 데이터가 로우 레벨로 된 데이터 세트만 적용할 수 있기 때문에 데이터의 첫 번째 칼럼을 사용자 아이디, 두 번째 칼럼을 아이템 아이디, 세 번째 칼럼을 평점으로 가정해 데이터를 로딩하고 네 번째 칼럼부터는 아예 로딩을 수행하지 않음

#### OS 파일 데이터를 Surprise 데이터 세트로 로딩
- 로딩되는 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있으면 안됨

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


https://grouplens.org/datasets/movielens/latest/

In [ ]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/ESAA/OB/Data/ml-latest-small/ratings.csv')
#ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('/content/drive/MyDrive/ESAA/OB/Data/ml-latest-small/ratings_noh.csv', index=False, header=False)

In [ ]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=', ', rating_scale=(0.5, 5))
data = Dataset.load_from_file('/content/drive/MyDrive/ESAA/OB/Data/ml-latest-small/ratings_noh.csv', reader=reader)

- Reader 클래스의 주요 생성 파라미터
    - line_format (string): 칼럼을 순서대로 나열하고 입력된 문자열을 공백으로 분리해 칼럼으로 인식함
    - sep (char): 칼럼을 분리하는 분리자이며, 디폴트는 '\t'임. 판다스 DataFrame에서 입력받을 경우에는 기재할 필요가 없음
    - rating_scale (tuple, optional): 평점 값의 최소 ~ 최대 평점을 설정함. 디폴트는 (1,5)이지만 ratings.csv 파일의 경우는 최소 평점이 0.5, 최대 평점이 5이므로 (0.5,5)로 설정하였음

In [ ]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

#수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)

#학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

#### 판다스 DataFrame에서 Surprise 데이터 세트로 로딩
- Dataset.load_from_df()를 이용하면 판다스의 DataFrame에서도 Surprise 데이터 세트로 로딩할 수 있음
- 주의할 점은 DataFrame 역시 사용자 아이디, 아이템 아이디, 평점 칼럼 순서를 지켜야 한다는 것임

In [ ]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/drive/MyDrive/ESAA/OB/Data/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5,5.0))

#ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 함
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

### Surprise 추천 알고리즘 클래스 <br>
- SVD: 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘
- KNNBasic: 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
- BaselineOnly: 사용자 Bias를 감안한 SGD 베이스라인 알고리즘
<br><br>
- Surprise SVD 비용 함수는 사용자 베이스라인(Baseline) 편향성을 감안한 평점 예측에 Regularization을 적용한 것임
    - 사용자 예측 Rating: 
$ \hat{rui} = μ + bu + bi + qTipu $
    - Regularization을 적용한 비용 함수: 
$ Σ(rui-\hat{rui})2 + λ(b2i+b2u+||qi||2+||pu||2) $
<br><br>
- SVD 클래스의 입력 파라미터
    - n_factors: 잠재 요인 K의 개수로 디폴트는 100이며며 커질수록 정확도가 높아질 수 있으나 과적합 문제가 발생할 수 있음
    - n_epochs: SGD (Stochastic Gradient Descent) 수행 시 반복 횟수로로 디폴트는 20
    - biased (bool): 베이스라인 사용자 편향 적용 여부로 디폴트는 True
- n_factors와 n_epochs 값을 변경해 튜닝할 수 있으나 튜닝 효과는 크지 않음'
- biased의 경우는 큰 이슈가 없는 한 디폴트인 True로 설정을 유지하는 것이 좋음
<br><br>
<img src='https://ifh.cc/g/zZpNKd.png' border='0'>
<br>
- SVD++ 알고리즘의 RMSE, MAE 성적이 갖아 좋지만, 상대적으로 시간이 너무 오래 걸려 데이터가 조금만 더 커져도 사용하기가 어려울 것으로 보임
- SVD++를 제외하면 SVD와 k-NN Baseline이 갖아 성능 평가 수치가 좋음
- k-NN 자체는 성능이 상대적으로 뒤지지만, Baseline을 결합한 경우 성능 평가 수치가 대폭 향상되었음
- Baseline이라는 의미는 각 개인이 평점을 부여하는 성향을 반영해 평점을 계산하는 방식을 말함


### 베이스라인 평점
- 개인의 성향을 반영해 아이템 평가에 편향성(bias) 요소를 반영하여 평점을 부과하는 것을 베이스라인 평점 (Baseline Rating)이라고 함
- 보통 베이스라인 평점은 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 공식으로 계산됨
    - 전체 평균 평점 = 모든 사용자의 아이템에 대한 평점을 평균한 값
    - 사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점
    - 아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균 평점
    <br><br> <img src='https://ifh.cc/g/RHbdXd.jpg' border='0'>


### 교차 검증과 하이퍼 파라미터 튜닝 <br>
- Surprise는 교차 검증과 하이퍼 파라미터 튜닝을 위해 사이킷런과 유사한 `cross_validate()`와 GridSearchCV 클래스를 제공함
- `cross_validate()` 함수는 surprise.model_selection 모듈 내에 존재하며, 폴드된 데이터 세트의 개수와 성능 측정 방법을 명시해 교차 검증을 수행함


In [ ]:
from surprise.model_selection import cross_validate

#판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('/content/drive/MyDrive/ESAA/OB/Data/ml-latest-small/ratings.csv') #reading data in pandas df
reader = Reader(rating_scale=(0.5,5.0))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=True)

- Surprise의 GridSearchCV도 사이킷런의 GridSearchCV와 유사하게 교차 검증을 통한 하이퍼 파라미터 최적화를 수행함
- 하이퍼 파라미터 최적화는 알고리즘의 유형에 따라 다를 수 있지만, SVD의 경우 주로 점진적 하강 방식 (Stochastic Gradient Descent)의 반복 횟수를 지정하는 n_epochs와 SVD의 잠재 요인 K의 크기를 지정하는 n_factors를 튜닝함

In [ ]:
from surprise.model_selection import GridSearchCV

#최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs': [20,40,60],
              'n_factors': [50,100,200]}

#CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
gs.fit(data)

#최고 RMSE Evaluation 점수와 그 때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

### Surprise를 이용한 개인화 영화 추천 시스템 구축 <br>
- Surprise를 이용해 잠재 요인 협업 필터링 기반의 개인화된 영화 추천을 구현함
- Surprise 패키지로 학습된 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은 영화 중에서 개인 취향에 가장 적절한 영화를 추천함
<br><br>
- Surprise는 데이터 세트를 train_test_split()을 이용해 내부에서 사용하는 TrainSet 클래스 객체로 변환하지 않으면 fit()을 통해 학습할 수가 없음

In [ ]:
#다음 코드는 train_test_split()으로 분리되지 않는 데이터 세트에 fit()을 호출해 오류가 발생함
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

In [ ]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=', ', rating_scale=(0.5,5))
#DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds(ratings_file='/content/drive/MyDrive/ESAA/OB/Data/ml-latest-small/ratings.csv', reader=reader)

#전체 데이터를 학습 데이터로 생성함
trainset = data_folds.build_full_trainset()

In [ ]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [ ]:
#영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('/content/drive/MyDrive/ESAA/OB/Data/ml-latest-small/movies.csv')

#userId=9의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
print(movies[movies['movieId']==42])

In [ ]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

In [ ]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()

    #모든 영화의 movieId를 리스트로 생성
    total_movies = movies['movieId'].tolist()

    #모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수:', len(seen_movies), ' 추천 대상 영화 수:', len(unseen_movies), ' 전체 영화 수:', len(total_movies))

unseen_movies = get_unseen_surprise(ratings, movies, 9)

In [ ]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    #알고리즘 객체의 predict() 메소드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId) for movieId in unseen_movies)]

    #predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음
    #[Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98), ...]

    #이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함
    #sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행
    def sortkey_est(pred):
        return pred.est
    
    #sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]

    #top_n으로 추출된 영화의 정보 추출, 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions ]
    top_movie_rating = [ pred.est for pred in top_predictions ]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_title, top_movie_rating) ]

    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
    print(top_movie[1], ':', top_movie[2])